In [ ]:
import socket
import numpy as np
import time

In [ ]:
HOST = 'localhost'
PORT = 5000
BLK_SIZE = 1024*16*1000
NBLK = 16
HDR_SIZE = 64

header = bytearray(HDR_SIZE)
buffer = bytearray(BLK_SIZE*4)
data = np.empty((1000*NBLK, 1024, 16), dtype='float32')

def readBlock(s):
    recv_size = s.recv_into(memoryview(header), HDR_SIZE)
    if recv_size != 64:
        print(f'Error receiving header, size={recv_size}')
        raise

    view = memoryview(buffer)
    next_offset = 0
    while BLK_SIZE*4 - next_offset > 0:
        recv_size = s.recv_into(view[next_offset:], BLK_SIZE*4 - next_offset)
        next_offset += recv_size
#        print(f'receive {recv_size} bytes')

In [ ]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))

    start = time.time()
    for n in range(NBLK):
        readBlock(s)
        data[n*1000:(n+1)*1000,:,:] = np.frombuffer(buffer, dtype='float32').reshape((1000, 1024, 16))
        print(f'received #{n} block, counter={header[:8].hex()}')
    end = time.time()

print(f'number of blocks: {NBLK},  time per block: {(end - start) / NBLK}s')
print(f'transfer rate: {NBLK*BLK_SIZE*4 / (end - start) * 10**(-9)} GB/s')

In [ ]:
header.hex()

In [ ]:
header[:5].hex()